# Chap08 文本数据

In [15]:
import numpy as np
import pandas as pd
import re

## str对象
### str对象的设计意图
1. str对象是定义在Index或Series上的属性，专门用于处理每个元素的文本内容。对一个序列进行文本处理，首先需要获取其str对象
2. python的标准库中也有str模块，而在许多函数的用法上pandas照搬了str模块的设计
### []索引器
1. str对象，可以理解为其对字符串进行了序列化的操作，在一般的字符串中，通过`[]`可以取出某个位置的元素，也可以通过**切片**得到子串，如果超出范围则返回缺失值
### string类型
1. 绝大多数对于object和string类型的序列使用str对象方法产生的结果是一致的，但也有存在较大差异的时候
   - 序列中至少存在一个可迭代对象（字符串，字典，列表）的情况下才使用str属性，对于一个可迭代对象，string类型的str对象和object类型的str对象返回的结果可能是不同的
   - string类型是Nullable类型，但object不是，也就是string类型的序列，如果调用的str方法返回值为整数Series和布尔Series时，其分别对应的dtype是Int和boolean的Nullable类型，而object类型则会分别返回int/float和bool/object，取决于缺失值的存在与否。同时，字符串的比较操作，也具有相似的特性，string返回Nullable类型，但object不会
2. 对于全体元素为数值类型的序列，即使其类型为object或者category也不允许直接使用str属性，如果需要把数字当成string类型处理，可以使用astype强制转化为string类型的Series

In [2]:
# str模块
var = 'abcd'
str.upper(var)

'ABCD'

In [3]:
# pandas照搬str模块的设计
s = pd.Series(['abcd', 'efg', 'hi'])
s.str.upper()

0    ABCD
1     EFG
2      HI
dtype: object

In [5]:
s.str[2]

0      c
1      g
2    NaN
dtype: object

In [6]:
s.str[-1:0:-2]

0    db
1     g
2     i
dtype: object

In [7]:
s = pd.Series([{1: 'temp_1', 2:'temp_2'}, ['a','b'], 0.5, 'my_string'])
s.str[1] # object类型

0    temp_1
1         b
2       NaN
3         y
dtype: object

In [8]:
s.astype('string').str[1] # string类型

0    1
1    '
2    .
3    y
dtype: string

In [9]:
s = pd.Series(['a'])
s == 'a'

0    True
dtype: bool

In [10]:
s.astype('string') == 'a'

0    True
dtype: boolean

In [12]:
s = pd.Series([12, 345, 6789])
s.astype('string').str[1]

0    2
1    4
2    7
dtype: string

## 正则表达式基础

In [14]:
dict_re = {'元字符':['.','[]','[^ ]','*','+','?','{n,m}','(xyz)','|','\\','^','$'],
           '描述':['匹配除换行符以外的任意字符',
                 '字符类，匹配方括号中包含的任意字符',
                 '否定字符类，匹配方括号中不包含的任意字符',
                 '匹配前面的子表达式零次或多次',
                 '匹配前面的子表达式一次或多次',
                 '匹配前面的子表达式零次或一次',
                 '花括号，匹配前面字符至少n次，但是不超过m次',
                 '字符组，按照确切的顺序匹配字符xyz',
                 '分支结构，匹配符号之前的字符或后面的字符',
                 '转义符，它可以还原元字符原来的含义',
                 '匹配行的开始',
                 '匹配行的结束']}
df_re = pd.DataFrame(dict_re)
df_re 

,元字符,描述
0,.,匹配除换行符以外的任意字符
1,[],字符类，匹配方括号中包含的任意字符
2,[^ ],否定字符类，匹配方括号中不包含的任意字符
3,*,匹配前面的子表达式零次或多次
4,+,匹配前面的子表达式一次或多次
5,?,匹配前面的子表达式零次或一次
6,"{n,m}",花括号，匹配前面字符至少n次，但是不超过m次
7,(xyz),字符组，按照确切的顺序匹配字符xyz
8,|,分支结构，匹配符号之前的字符或后面的字符
9,\,转义符，它可以还原元字符原来的含义


In [28]:
dict_str = {'简写':['\\w','\\W','\\d','\\D','\\s','\\S','\\B'],
            '描述':['匹配所有字母、数字、下划线:[a-zA-Z0-9]',
                  '匹配非字母和数字的字符:[^\w]',
                  '匹配数字:[0-9]',
                  '匹配非数字:[^\d]',
                  '匹配空格符:[\\t\\n\\f\\r\\p{Z}]',
                  '匹配非空格符:[^\s]',
                  '匹配一组非空字符开头或结尾的位置，不代表具体字符']}
df_str = pd.DataFrame(dict_str)
df_str

,简写,描述
0,\w,匹配所有字母、数字、下划线:[a-zA-Z0-9]
1,\W,匹配非字母和数字的字符:[^\w]
2,\d,匹配数字:[0-9]
3,\D,匹配非数字:[^\d]
4,\s,匹配空格符:[\t\n\f\r\p{Z}]
5,\S,匹配非空格符:[^\s]
6,\B,匹配一组非空字符开头或结尾的位置，不代表具体字符


In [17]:
re.findall(r'.', 'abc')

['a', 'b', 'c']

In [24]:
re.findall(r'[ac]{3}','abcccac')

['ccc']

In [29]:
re.findall(r'.s','Apple! This Is an Apple!')

['is', 'Is']

## 文本处理的五类操作
### 拆分
1. `str.split(正则表达式,n,expand=True)`
   - `n`表示从左到右的最大拆分次数
   - `expand`表示是否展开为多个列
2. `str.rsplit(正则表达式,n,expand=True)`（疑似存在bug无法使用）
   - `n`表示从右到左的最大拆分次数
### 合并
1. `str.join`
   - 用某个连接符把Series中的字符串列表连接起来
   - 列表中出现了非字符串元素则返回缺失值
2. `str.cat`
   - 用于合并两个序列
   - 主要参数
     - `sep`连接符
     - `join`连接形式，默认为以索引为键的左连接
     - `na_rep`缺失值替代符号
### 匹配
1. `str.contains(正则表达式)`
   - 返回了每个字符串是否包含正则模式的布尔序列
   - 检测字符串开头和结尾的模式可以采用contains和`^$`搭配使用
2. `str.startswith('字符串')`
   - 返回了每个字符串以给定模型为开始的布尔序列，不支持正则表达式
3. `str.endswitch('字符串')`
   - 返回了每个字符串以给定模型为结束的布尔序列，不支持正则表达式
4. `str.match(正则表达式)`
   - 用正则表达式来检测开始或结束字符串的模式
   - 若要检测结束字符串，需要先对字符串进行反转`s.str[::-1]`，即`s.str[::-1].str.match(正则表达式)`
5. 返回索引的匹配函数
   - `str.find`返回从左到右第一次匹配的位置的索引，未找到则返回-1
   - `str.rfind`返回从右到左第一次匹配的位置的索引，未找到则返回-1
   - 这两个函数都不支持正则匹配，只能用于字符子串的匹配
### 替换`str.replace`
1. 可以利用子组的方法，通过传入自定义的替换函数来分别进行处理实现对不同部分的有差别替换
2. 还可以使用命名子组`?P<子组名>`
### 提取
1. `str.extract`
   - 可以通过子组的命名，直接对新生成DataFrame的列命名
2. `str.extractall`
   - 把所有符合条件的模式全部匹配出来，如果存在多个结果，则以多级索引的方式存储
3. `str.findall`
   - 功能类似extractall
   - 该函数将结果存入列表中，每个行只对应一组匹配

In [30]:
s = pd.Series(['上海市黄浦区方浜中路249号','上海市宝山区密山路5号'])
s.str.split('[市区路]')

0    [上海, 黄浦, 方浜中, 249号]
1       [上海, 宝山, 密山, 5号]
dtype: object

In [31]:
s.str.split('[市区路]', n = 2, expand=True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


In [32]:
s = pd.Series([['a','b'], [1,'a'], [['a','b'],'c']])
s

0         [a, b]
1         [1, a]
2    [[a, b], c]
dtype: object

In [34]:
s.str.join('-')

0    a-b
1    NaN
2    NaN
dtype: object

In [36]:
s1 = pd.Series(['a','b'])
s2 = pd.Series(['cat','dog','duck'])
s1.str.cat(s2, sep='-', join = 'right', na_rep = '?')

0     a-cat
1     b-dog
2    ?-duck
dtype: object

In [40]:
s = pd.Series(['my cat', 'he is fat', ' 2at', 'railway station'])
s.str.contains('\s\wat')

0     True
1     True
2     True
3    False
dtype: bool

In [41]:
s.str.startswith('m')

0     True
1    False
2    False
3    False
dtype: bool

In [42]:
s.str.endswith('t')

0     True
1     True
2     True
3    False
dtype: bool

In [43]:
s.str.match('m|h')

0     True
1     True
2    False
3    False
dtype: bool

In [46]:
s.str[::-1]

0             tac ym
1          taf si eh
2               ta2 
3    noitats yawliar
dtype: object

In [45]:
s.str[::-1].str.match('ta[f|g]|n')

0    False
1     True
2    False
3     True
dtype: bool

In [49]:
s = pd.Series(['上海市黄浦区方浜中路249号',
               '上海市宝山区密山路5号',
               '北京市昌平区北农路2号'])
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
city = {'上海市':'shanghai','北京市':'beijing'}
district = {'昌平区':'cp district','黄浦区':'hp district','宝山区':'bs district'}
road = {'方浜中路':'Mid Fangbang Road',
        '密山路':'Mishan Road',
        '北农路':'Beinong Road'}
def my_func(m):
    str_city = city[m.group(1)]
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No.' + m.group(4)[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])
s.str.replace(pat, my_func, regex=True)

0    shanghai hp district Mid Fangbang Road No.249
1            shanghai bs district Mishan Road No.5
2            beijing cp district Beinong Road No.2
dtype: object

In [50]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [51]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [52]:
s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])
s

my_A    A135T15,A26S5
my_B     B674S2,B25T6
dtype: object

In [53]:
pat = '[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [54]:
s.str.extract(pat)

,0,1
my_A,135,15
my_B,674,2


In [55]:
s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

## 常用字符串函数
### 字母型函数
1. `upper`全部转化为大写字母
2. `lower`全部转化为小写字母
3. `title`每个单词的首字母大写
4. `capitablize`只有第一个单词的首字母大写
5. `swapcase`将单词中的大写字母转为小写，小写字母转为大写
### 数值型函数
1. `pd.to_numeric`
   - 不是str对象上的方法，但是能够对字符格式的数值进行快速转换和筛选
   - 主要参数
     - `errors`非数值（不能转换为数值）的处理模式
       - `raise`直接报错
       - `coerce`设为缺失，在数据清洗时，可以利用该值快速查看非数值型的行`s[pd.to_numeric(s, errors='coerce).isna()]`
       - `ignore`保持原来的字符串
     - `downcast`转换类型
### 统计型函数
1. `count`返回出现正则模式的次数
2. `len`返回字符串的长度
### 格式型函数
1. 除空型
   - `strip`去除两侧空格
   - `rstrip`去除右侧空格
   - `lstrip`去除左侧空格
2. 填充型
   - `str.pad(填充后字符串长度,填充的方向,填充内容)`
   - `str.rjust`等价于pad函数的填充方向为**left**，也等价于`zfill`函数
   - `str.ljust`等价于pad函数的填充方向为**right**
   - `str.center`等价于pad函数的填充方向为**both**

In [56]:
s = pd.Series(['cat rat fat at','get feed sheet heat'])
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [57]:
s = pd.Series(['a','b','c'])
s.str.pad(5,'left','*')

0    ****a
1    ****b
2    ****c
dtype: object

In [58]:
s.str.rjust(5,'*')

0    ****a
1    ****b
2    ****c
dtype: object